In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
import inspect
# from scipy import linalg
import numpy as np
print(inspect.getsource(np.linalg.svd))

@array_function_dispatch(_svd_dispatcher)
def svd(a, full_matrices=True, compute_uv=True, hermitian=False):
    """
    Singular Value Decomposition.

    When `a` is a 2D array, and ``full_matrices=False``, then it is
    factorized as ``u @ np.diag(s) @ vh = (u * s) @ vh``, where
    `u` and the Hermitian transpose of `vh` are 2D arrays with
    orthonormal columns and `s` is a 1D array of `a`'s singular
    values. When `a` is higher-dimensional, SVD is applied in
    stacked mode as explained below.

    Parameters
    ----------
    a : (..., M, N) array_like
        A real or complex array with ``a.ndim >= 2``.
    full_matrices : bool, optional
        If True (default), `u` and `vh` have the shapes ``(..., M, M)`` and
        ``(..., N, N)``, respectively.  Otherwise, the shapes are
        ``(..., M, K)`` and ``(..., K, N)``, respectively, where
        ``K = min(M, N)``.
    compute_uv : bool, optional
        Whether or not to compute `u` and `vh` in addition to `s`.  True
 

In [2]:
#  Download and load CIFAR-10 dataset
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

def compute_svd_singular_values(image_batch):
    singular_values = []
    for image in image_batch:
        squeeze = image.squeeze()
        U, S, V = torch.svd(squeeze)
        singular_values.append(S)
    return torch.stack(singular_values)

# Compute singular values for train and test sets
train_singular_values = [compute_svd_singular_values(batch[0]).to(device) for batch in trainloader]
test_singular_values = [compute_svd_singular_values(batch[0]).to(device) for batch in testloader]

Files already downloaded and verified
Files already downloaded and verified


In [3]:
len(train_singular_values[0])

64

### Step 2: Neural Network Training

#### A. Define the Neural Network Model

In [4]:
class SVDPredictor(nn.Module):
    def __init__(self):
        super(SVDPredictor, self).__init__()
        self.fc1 = nn.Linear(32*32, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 32)  # Predicting 32 singular values

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        return self.fc3(x)

# move the model to gpu
model = SVDPredictor().to(device)

#### B. Define Loss Function and Optimizer

In [5]:
import torch.optim as optim

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)


#### C. Training the Model

In [6]:
# Training loop
num_epochs = 50  # Set the number of epochs
print_every_n_batches = 100  # Print log after every 100 batches

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, _) in enumerate(trainloader):
        images = images.view(images.shape[0], -1).to(device)
        labels = train_singular_values[i]  # Precomputed singular values

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Log the running loss
        if (i + 1) % print_every_n_batches == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(trainloader)}], Loss: {running_loss / print_every_n_batches:.4f}')
            running_loss = 0.0

print('Finished Training')



Epoch [1/50], Step [100/782], Loss: 0.8692
Epoch [1/50], Step [200/782], Loss: 0.7920
Epoch [1/50], Step [300/782], Loss: 0.7587
Epoch [1/50], Step [400/782], Loss: 0.7413
Epoch [1/50], Step [500/782], Loss: 0.7311
Epoch [1/50], Step [600/782], Loss: 0.7219
Epoch [1/50], Step [700/782], Loss: 0.7093
Epoch [2/50], Step [100/782], Loss: 0.6949
Epoch [2/50], Step [200/782], Loss: 0.6863
Epoch [2/50], Step [300/782], Loss: 0.6813
Epoch [2/50], Step [400/782], Loss: 0.6758
Epoch [2/50], Step [500/782], Loss: 0.6714
Epoch [2/50], Step [600/782], Loss: 0.6666
Epoch [2/50], Step [700/782], Loss: 0.6562
Epoch [3/50], Step [100/782], Loss: 0.6487
Epoch [3/50], Step [200/782], Loss: 0.6383
Epoch [3/50], Step [300/782], Loss: 0.6356
Epoch [3/50], Step [400/782], Loss: 0.6271
Epoch [3/50], Step [500/782], Loss: 0.6248
Epoch [3/50], Step [600/782], Loss: 0.6190
Epoch [3/50], Step [700/782], Loss: 0.6069
Epoch [4/50], Step [100/782], Loss: 0.5945
Epoch [4/50], Step [200/782], Loss: 0.5842
Epoch [4/50

### Step 3: Evaluation


##### Implementing NMSE and MAE Computation

In [7]:
def compute_nmse(outputs, labels):
    mse = ((outputs - labels) ** 2).mean(axis=1)
    norm = (labels ** 2).mean(axis=1)
    nmse = mse / norm
    return nmse

def compute_mae(outputs, labels):
    mae = torch.abs(outputs - labels).mean(axis=1)
    return mae


In [8]:
model.eval()  # Set the model to evaluation mode

total_nmse = []
total_mae = []
total_samples = 0

with torch.no_grad():
    for i, (images, _) in enumerate(testloader):
        images = images.view(images.shape[0], -1).to(device)
        labels = test_singular_values[i]  # Use precomputed singular values

        outputs = model(images)
        nmse = compute_nmse(outputs, labels)
        mae = compute_mae(outputs, labels)

        total_nmse.extend(nmse.tolist())
        total_mae.extend(mae.tolist())
        total_samples += images.size(0)




Average NMSE: 0.0876
Average MAE: 0.2137
